In [106]:
import matplotlib.pyplot as plt
import gym
import sys

import torch
from torch import nn 
from torch import optim

import numpy as np

print("PyTorch:\t{}".format(torch.__version__))

class policy_estimator():
    def __init__(self, env):
        self.n_input = env.observation_space.shape[0]
        self.n_outputs = env.action_space.n
        
        #Define network
        self.network = nn.Sequential(
            nn.Linear(self.n_input, 16),
            nn.ReLU(), 
            nn.Linear(16, self.n_outputs),
            nn.Softmax(dim=-1)
        )
        
    def predict(self, state):
        action_probs = self.network(torch.FloatTensor(state))
        return action_probs
                            

PyTorch:	2.0.0+cu117


In [107]:
def discount_rewards(rewards, gamma=0.99):
            r = np.array([gamma**i * rewards[i] for i in range(len(rewards))])
            #Reverse the array direction for cumsum and then revert back to the original order
            r = r[::-1].cumsum()[::-1]
            return r - r.mean()

In [115]:
def reinforce(env, policy_estimator, num_episodes=2000,
                      batch_size=10, gamma=0.99):
    #Set up lists to hold results
    total_rewards = []
    batch_rewards = []
    batch_actions = []
    batch_states = []
    batch_counter = 1
            
    # Define optimlizer     
    optimizer = optim.Adam(policy_estimator.network.parameters(), lr=0.01)
            
    action_space = np.arange(env.action_space.n)
    ep = 0
    while ep < num_episodes:
        s_0 = env.reset()
        states = {agent_id: list() for agent_id in s_0.keys()}     
        rewards = {agent_id: list() for agent_id in s_0.keys()}
        actions = {agent_id: list() for agent_id in s_0.keys()}
        done = False
                
        while not done:
            #Get action and convert it to joint_action
            joint_action = {agent_id: {'movement':None, 'ask_communication': 1} for agent_id in s_0.keys()}
            for agent_id in s_0.keys():
                action_probs = policy_estimator.predict(s_0[agent_id]).detach().numpy()
                joint_action[agent_id]['movement'] = np.random.choice(action_space, p=action_probs)
                    
                s_1, r_0, dones, _ = env.step(joint_action)

                
                {k:v.append(s_0[k]) for k,v in states.items()}
                {k:v.append(r_0[k]) for k,v in rewards.items()}
                {k:v.append(joint_action[k]['movement']) for k,v in actions.items()}
                s_0 = s_1
                    
                done = False not in dones.values()
                # If done, batch data
                if done:
                    for agent_id in s_0.keys():
                        if batch_counter < batch_size:
                            batch_rewards.extend(discount_rewards(rewards[agent_id], gamma))
                            batch_states.extend(states[agent_id])
                            batch_actions.extend(actions[agent_id])
                            batch_counter += 1
                                
                            total_rewards.append(sum(rewards[agent_id]))
                            
                    #If batch is complete, update network
                    if batch_counter == batch_size:
                        optimizer.zero_grad()
                        state_tensor = torch.FloatTensor(batch_states)
                        reward_tensor = torch.FloatTensor(batch_rewards)
                        # Actions are used as indices, must be LongTensor
                        action_tensor = torch.LongTensor(batch_actions)
                            
                        #Calculate loss
                        logprob = torch.log(policy_estimator.predict(state_tensor))
                        selected_logprobs = reward_tensor * torch.gather(logprob, 1, action_tensor.unsqueeze(1)).squeeze()
                        loss = -selected_logprobs.mean()
                            
                        # Calculate gradients
                        loss.backward()
                        # applu gradients
                        optimizer.step()
                            
                        batch_rewards = []
                        batch_actions = []
                        batch_states = []
                        batch_counter = 1
                            
                        avg_rewards = np.mean(total_rewards[-100:])
                        # Print running average
                        print("\rEp: {} Average of last 100:" +   
                            "{:.2f}".format(
                            ep + 1, avg_rewards), end="")
                        ep += 1
    return total_rewards

In [116]:
env = gym.make('CartPole-v0')
policy_est = policy_estimator(env)
rewards = reinforce(env, policy_est)

Ep: {} Average of last 100:2000.00

In [119]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

class VGGBLOCK(Module):
    def __init_(self, numChannels, classes):
        super(VGGBLOCK,self).__init__()
        
        self.conv1 = Conv2d(in_channels=6, out_channels=128, kernel_size = (3,3), stride=1, padding=1)
        self.conv2 = Conv2d(in_channels=128, out_channels=128, kernel_size = (3,3), stride=1, padding=1)
        self.conv3 = Conv2d(in_channels=128, out_channels=128, kernel_size = (3,3), stride=1, padding=1)
        self.pool1 = MaxPool2d(size=2, stride=1)
    
    def forwar
        
        

In [25]:
import torch.nn as nn
m = nn.Conv2d(in_channels=6, out_channels=1, kernel_size=3, stride=1,padding=1)
input = torch.randn(6,5,5)
m(input)

tensor([[[ 0.2451, -0.2843, -0.3353,  0.0593,  0.3627],
         [-0.4267,  0.0633, -0.6618,  0.3487, -0.1211],
         [ 0.0750, -0.4737,  0.3125,  0.7875, -0.3587],
         [ 0.8480,  0.0339,  0.4692,  0.0946, -0.4433],
         [ 0.1824, -0.0720,  0.0922, -0.2039,  0.6299]]],
       grad_fn=<SqueezeBackward1>)